### Нужно написать две программы:

1) Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian).

2) Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии  
1. Простое последовательное чтение 
2. Многопоточная + memory-mapped files. 

Сравните время работы.

In [1]:
import random
import numpy as np
import time

import mmap
import os

from concurrent.futures import ThreadPoolExecutor, Future
from multiprocessing import Process, Pool
import concurrent.futures

import warnings
warnings.filterwarnings('ignore')

In [2]:
length = 1000000000
f_name = "big_array_32.txt"

def print_res(sum_arr, max_arr, min_arr):
    print(f'Сумма элементов: \t{sum_arr}')
    print(f'Максимальный элемент: \t{max_arr}')
    print(f'Минимальный элемент: \t{min_arr}')

def time_counter(func):
    start_time = time.clock()
    func()
    print(time.clock() - start_time, "seconds")
    print("---------------------------------------")

### Генерация файла

In [8]:
CHUNK_SIZE = length
with open(f_name, 'wb') as f:
    arr = np.random.randint(2, length, size=CHUNK_SIZE//4, dtype=np.dtype('uint32').newbyteorder('B')).byteswap()                        
    f.write(arr.data)

### Считывание файла

In [9]:
# с memory-mapped files
with open(f_name, 'r+b') as f:
    with mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_WRITE) as mm:
        # восстанавливаем массив
        arr = np.frombuffer(mm, dtype=np.int64)
        print(len(arr))
        print(arr)

12500000
[ 3356088821434179587  -316720921354585599  4443556422207862784 ...
  7782215778208067331  3480084058049985797 -3387881190111985407]


In [10]:
# обычное
with open(f_name, 'r+b') as f:
    buf = f.read()
    array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))
    
    print(array)

[56639092  3773230 29014277 ... 62802736 29435362 30735312]


## Последовательное чтение и расчет величин

In [11]:
def simple_find():
    with open(f_name, 'r+b') as f:
        buf = f.read()
        array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))

        sum_num, max_num, min_num = 0, 0, 10**32
        mas = []

        for num in array:
            if (num > max_num):
                max_num = num
            if (num < min_num):
                min_num = num
            sum_num += num

        f.close()

    print_res(sum_num, max_num, min_num)
    
simple_find()

Сумма элементов: 	1250004595991438
Максимальный элемент: 	99999996
Минимальный элемент: 	6


## Многопоточное и memory-mapped files

### Только numpy

In [16]:
def numpy_find():
    with open(f_name, 'r+b') as f:
        buf = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
        array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))
        print_res(sum(array), array.max(), array.min())
        
numpy_find()

Сумма элементов: 	1250004595991438
Максимальный элемент: 	99999996
Минимальный элемент: 	6


### Многопоточная версия с future 

In [18]:
sum_num = 0
max_num = 0
min_num = 10**32

def search(array): 
    global sum_num
    global max_num
    global min_num
    
    for num in array:
        sum_num += num
        if num > max_num:
            max_num = num
        if num < min_num:
            min_num = num

def mmp_thread_future_find():
    chunk = length // 8

    global sum_num
    sum_num = 0
    global max_num
    max_num = 0
    global min_num
    min_num = 10**32
    
    with open(f_name, 'r+b') as f:
        buf = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
        array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))

        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            arrays = [array[0: chunk], array[chunk: 2*chunk], array[2*chunk: 3*chunk], array[3*chunk:4*chunk], \
                     array[4*chunk: 5*chunk], array[5*chunk: 6*chunk], array[6*chunk: 7*chunk], array[7*chunk:]]
            future = {executor.submit(search, ar): ar for ar in arrays}
            
    print_res(sum_num, max_num, min_num)
    
mmp_thread_future_find()

Сумма элементов: 	864199213411575
Максимальный элемент: 	99999996
Минимальный элемент: 	6


### Замер времени

In [19]:
print("Обычное решение с последовательным сравнением")
time_counter(simple_find)

print("Решение через методы numpy")
time_counter(numpy_find)

print("Решение с пулом потоков и маппингом")
time_counter(mmp_thread_future_find)

Обычное решение с последовательным сравнением
Сумма элементов: 	1250004595991438
Максимальный элемент: 	99999996
Минимальный элемент: 	6
9.314176299999986 seconds
---------------------------------------
Решение через методы numpy
Сумма элементов: 	1250004595991438
Максимальный элемент: 	99999996
Минимальный элемент: 	6
6.8776567000000455 seconds
---------------------------------------
Решение с пулом потоков и маппингом
Сумма элементов: 	829852926764703
Максимальный элемент: 	99999996
Минимальный элемент: 	6
11.394280999999978 seconds
---------------------------------------


## Прочие попытки, которые не привели к успеху

### Пул процессов
Этот пример бесконечно грузится

In [ ]:
sum_num = 0
max_num = 0
min_num = 10**32

lock_min = threading.Lock()
lock_max = threading.Lock()

chunk = length // 4

def search(array): 
    global sum_num
    global max_num
    global min_num
    
    for num in array:
        sum_num += num
        if num > max_num:
            max_num = num
        if num < min_num:
            min_num = num
    
with open(f_name, 'r+b') as f:
    buf = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
    array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))

    with Pool(4) as pool:    
        pool.map(search, [array[0: chunk], array[chunk: 2*chunk], array[2*chunk: 3*chunk], array[3*chunk:]])
    
    #num_processors = 4
    #p = Pool(processes = num_processors)
    #output = p.map(search,[array[0: chunk], array[chunk: 2*chunk], array[2*chunk: 3*chunk], array[3*chunk:]])
    #print(output)
    
print_res(sum_num, max_num, min_num)

### Пул потоков
Блокировки почему-то не работают, ответы выходят разные

In [8]:
sum_num = 0
max_num = 0
min_num = 10**32

lock_min = threading.Lock()
lock_max = threading.Lock()

chunk = length // 4

def search(array): 
    global sum_num
    global max_num
    global min_num
    
    global lock_max
    global lock_min
    
    for num in array:
        sum_num += num
        if num > max_num:
            with lock_max:
                max_num = num
        if num < min_num:
            with lock_min:
                min_num = num
            
with open(f_name, 'r+b') as f:
    buf = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
    array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))

    executor = ThreadPoolExecutor(max_workers=4)
    future1 = executor.submit(search, array[0: chunk]) 
    future2 = executor.submit(search, array[chunk: 2*chunk])
    future3 = executor.submit(search, array[2*chunk: 3*chunk])
    future4 = executor.submit(search, array[3*chunk:])
    
print_res(sum_num, max_num, min_num)

Сумма элементов: 	512432016046
Максимальный элемент: 	9999932
Минимальный элемент: 	74
